In [ ]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
mapbox_access_token = open('mapbox_token.txt').read()

In [ ]:
collected = pd.read_csv('Collected.csv')
print(collected.head())

In [ ]:
# Create a dictionary of english name : korean name
en_name = ['Im Ok-in', 'Shim Hoon', 'Lee Kwang-soo', 'Kim Myung-soon', 'Im Soon-deuk', 'Kim Nam-cheon', 'Mo Yoon-sook', 'Ju Yo-han', 'Lee Sun-hee', 'Cha Mi-ri-sa', 'Yeom Sang-seob', 'No Chun-myeong', 'Kang Kyung-ae', 'Baek Shin-ae', 'Han Mu-sook', 'Park Hwa-seong', 'Choi Jeong-hee', 'Choi Seung-hee', 'Na Hye-seok', 'Lee Sang', 'Kim Sa-ryang', 'Ji Ha-ryeon']
ko_name = list(collected['Name_ko'].unique())
ko_en_name = {key:value for key, value in zip(ko_name, en_name)}

In [ ]:
# Add a column of full english names 'Full_Name_en'
collected['Full_Name_en'] = collected['Name_ko'].apply(lambda x: ko_en_name.get(x))
print(collected.head())

In [ ]:
#Update Collected.csv
#collected.to_csv('Collected.csv')

In [ ]:
# Create dictionary of author names for dropdown
authors_collected = [{'label' : name, 'value': name} for name in collected['Full_Name_en'].unique()]

In [ ]:
#for whatever authors are selected, create a list of random colors generated
#make sure that the colors don't overlap
#random color generator? 
#print(index) before the graph with the colors and 
# marker(color - list I'd have created)

In [ ]:
# random color generator 
import random 

# generate random colors
def color_generator(authors):
    color_dict = {}
    for i in range(len(authors)):
        r = random.randint(0,255)
        b = random.randint(0,255)
        g = random.randint(0,255)
        color = 'rgb({r},{b},{g})'.format(r=r,b=b,g=g)
        if color not in color_dict:
            color_dict[authors[i]] = color
    return color_dict

In [ ]:
# Created 'colors' column in Collected.csv
color_dict = color_generator(list(collected['Full_Name_en'].unique()))
collected['colors'] = collected['Full_Name_en'].apply(lambda x: color_dict.get(x))
#collected.to_csv('Collected.csv')

In [ ]:
# Dash App
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)

In [ ]:
# nahyeseok dataframe
nahyeseok = collected[collected['Full_Name_en'] == 'Na Hye-seok']

# scattermapbox of nahyeseok
fig = go.Figure(go.Scattermapbox(
                      lat = nahyeseok['Latitude'],
                      lon = nahyeseok['Longitude'],
                      mode = 'markers',
                      marker = {
                          'color' : nahyeseok['colors'],
                          'size' : 9
                      },
                      hoverinfo = 'text',
                      hovertext = nahyeseok['Location']))

fig.update_layout(
            title = '20th Century Women Writers Itinerary',
            hovermode = 'closest',
            mapbox = dict(
                accesstoken=mapbox_access_token,
                style = 'light', 
                center = dict(lat = 0, lon = 0),
                zoom = 1
            ))

In [ ]:
# app layout using scattermapbox 
app.layout = html.Div([
    html.Div([dcc.Dropdown(
        id = 'author_dropdown',
        value = 'Na Hye-seok',
        multi = True,
        options = authors_collected)]),
    html.Div([
    html.Ul(id='index', children=[
        html.Li("Na Hye-seok", style={'color': collected.loc[collected['Full_Name_en'] == 'Na Hye-seok', 'colors'].values[0]})]),
    dcc.Graph(id = 'map', 
              figure = fig,
              config={'displayModeBar': False, 'scrollZoom': True}),
    html.Button('Next ▶', id='next_button')])
    ])

In [ ]:
@app.callback(
    Output(component_id = 'index', component_property = 'children'),
    [Input(component_id = 'author_dropdown', component_property = 'value')]
)
def update_index(authors):
    return [html.Li(x, style={'color': collected.loc[collected['Full_Name_en'] == x, 'colors'].values[0]})
            for x in authors]

In [ ]:
# specify port if error about port number being used already 
if __name__ == '__main__':
    app.run_server(port = 8056)